In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import scale
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

/home/antoine/anaconda3/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/home/antoine/anaconda3/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
train_fname = 'data/train.csv'
test_fname = 'data/test.csv'
df = pd.read_csv(train_fname, sep=';')
df_test = pd.read_csv(test_fname, sep=';')
(n_samples,n_variables) = df.shape

In [3]:
f_con = ['APP_NB','APP_NB_PAYS','APP_NB_TYPE','NB_CLASSES','NB_ROOT_CLASSES','NB_SECTORS','NB_FIELDS','INV_NB',
        'INV_NB_PAYS','INV_NB_TYPE','cited_n','cited_nmiss','cited_age_min','cited_age_median','cited_age_max','cited_age_mean',
        'cited_age_std','NB_BACKWARD_NPL','NB_BACKWARD_XY','NB_BACKWARD_I','NB_BACKWARD_AUTRE','NB_BACKWARD_PL',
        'NB_BACKWARD','pct_NB_IPC','pct_NB_IPC_LY','oecd_NB_ROOT_CLASSES','oecd_NB_BACKWARD_PL','oecd_NB_BACKWARD_NPL',
        'IDX_ORIGIN','IDX_RADIC','PRIORITY_MONTH','FILING_MONTH','PUBLICATION_MONTH','BEGIN_MONTH']
f_cat = ['VOIE_DEPOT','COUNTRY','SOURCE_BEGIN_MONTH','FISRT_APP_COUNTRY','FISRT_APP_TYPE','LANGUAGE_OF_FILLING',
        'FIRST_CLASSE','TECHNOLOGIE_SECTOR','TECHNOLOGIE_FIELD','MAIN_IPC','FISRT_INV_COUNTRY','FISRT_INV_TYPE','SOURCE_CITED_AGE',
        'SOURCE_IDX_ORI','SOURCE_IDX_RAD']

## 1) Données continues.

In [4]:
X_train_con = df[f_con].values
X_test_con = df_test[f_con].values

In [5]:
for i in range(n_samples):
    for j in range(30,34):
        if(isinstance(X_train_con[i][j],basestring)):
            str = X_train_con[i][j].split('/')
            X_train_con[i][j] = int(str[0]) + 12*int(str[1])

for i in range(X_test_con.shape[0]):
    for j in range(30,34):
        if(isinstance(X_test_con[i][j],basestring)):
            str = X_test_con[i][j].split('/')
            X_test_con[i][j] = int(str[0]) + 12*int(str[1])

NameError: name 'basestring' is not defined

In [ ]:
imputer = Imputer()
X_train_con = imputer.fit_transform(X_train_con)
X_test_con = imputer.fit_transform(X_test_con)

In [ ]:
scale(X_train_con, copy=False);
scale(X_test_con, copy=False);

In [8]:
pd.DataFrame(data=X_train_con, columns=list(f_con)).to_csv(path_or_buf='data/train_con_1.csv', sep=';')
pd.DataFrame(data=X_test_con, columns=list(f_con)).to_csv(path_or_buf='data/test_con_1.csv', sep=';')

## 2) Données catégorielles

In [6]:
class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [34]:
X_train_cat = df[f_cat].values
X_test_cat = df_test[f_cat].values

In [35]:
encoder = MultiColumnLabelEncoder()
X_cat = encoder.fit_transform(pd.DataFrame(np.concatenate([X_train_cat,X_test_cat], axis=0),columns=f_cat)).values

In [36]:
for i in range(len(f_cat)):
    print("{} : {} {}".format(i,f_cat[i],np.max(X_cat[:,i])))

0 : VOIE_DEPOT 1
1 : COUNTRY 94
2 : SOURCE_BEGIN_MONTH 2
3 : FISRT_APP_COUNTRY 143
4 : FISRT_APP_TYPE 4
5 : LANGUAGE_OF_FILLING 29
6 : FIRST_CLASSE 41178
7 : TECHNOLOGIE_SECTOR 4
8 : TECHNOLOGIE_FIELD 34
9 : MAIN_IPC 629
10 : FISRT_INV_COUNTRY 150
11 : FISRT_INV_TYPE 4
12 : SOURCE_CITED_AGE 1
13 : SOURCE_IDX_ORI 1
14 : SOURCE_IDX_RAD 1


In [37]:
X_train_cat = X_cat[0:n_samples,:]
X_test_cat = X_cat[n_samples:,:]

In [38]:
ohencoder = OneHotEncoder()
X_train_cat_ohe = ohencoder.fit_transform(X_train_cat)
X_test_cat_ohe = ohencoder.transform(X_test_cat)

In [41]:
X_train_cat_ohe

<259431x37531 sparse matrix of type '<type 'numpy.float64'>'
	with 3891465 stored elements in Compressed Sparse Row format>

In [40]:
X_train_cat_ohe.dump('data/train_cat_ohe_1.csv')
X_test_cat_ohe.dump('data/test_cat_ohe_1.csv')

AttributeError: dump not found

In [138]:
pd.DataFrame(data=X_train_cat, columns=list(f_cat)).to_csv(path_or_buf='data/train_cat_1.csv', sep=';')
pd.DataFrame(data=X_test_cat, columns=list(f_cat)).to_csv(path_or_buf='data/test_cat_1.csv', sep=';')

## 3) Recollement pour certains modèles

In [142]:
X_train = np.concatenate((X_train_con,X_train_cat), axis=1)
X_test = np.concatenate((X_test_con,X_test_cat), axis=1)

pd.DataFrame(data=X_train, columns=f_con + f_cat).to_csv(path_or_buf='data/train_1.csv', sep=';')
pd.DataFrame(data=X_test, columns=f_con + f_cat).to_csv(path_or_buf='data/test_1.csv', sep=';')